In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
import torch
import re
import json
import os

# Load model and tokenizer
model_name = "nvidia/AceMath-1.5B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype="auto", device_map="auto")

# Load GSM8K dataset
gsm8k = load_dataset("gsm8k", "main")
test_set = gsm8k["test"]

def download_gsm8k():
    import requests
    import os
    
    # Create data directory if it doesn't exist
    os.makedirs("gsm8k_data", exist_ok=True)
    
    # URLs for train and test data
    train_url = "https://raw.githubusercontent.com/openai/grade-school-math/master/grade_school_math/data/train.jsonl"
    test_url = "https://raw.githubusercontent.com/openai/grade-school-math/master/grade_school_math/data/test.jsonl"
    
    # Download train set
    train_response = requests.get(train_url)
    with open("gsm8k_data/train.jsonl", "wb") as f:
        f.write(train_response.content)
    
    # Download test set
    test_response = requests.get(test_url)
    with open("gsm8k_data/test.jsonl", "wb") as f:
        f.write(test_response.content)
    
    print("GSM8K dataset downloaded successfully.")

# Function to load the GSM8K dataset from files
def load_gsm8k_from_file():
    train_data = []
    test_data = []
    
    # Check if files exist, download if not
    if not (os.path.exists("gsm8k_data/train.jsonl") and os.path.exists("gsm8k_data/test.jsonl")):
        download_gsm8k()
    
    # Load train data
    with open("gsm8k_data/train.jsonl", "r") as f:
        for line in f:
            train_data.append(json.loads(line))
    
    # Load test data
    with open("gsm8k_data/test.jsonl", "r") as f:
        for line in f:
            test_data.append(json.loads(line))
    
    print(f"Loaded {len(train_data)} training examples and {len(test_data)} test examples")
    return train_data, test_data

# Function to extract the numerical answer
def extract_answer(text):
    if "####" in text:
        answer_part = text.split("####")[-1].strip()
        numbers = re.findall(r'[-+]?\d*\.?\d+', answer_part)
        if numbers:
            return float(numbers[0])
    
    # Fallback: extract the last number in the full text
    numbers = re.findall(r'[-+]?\d*\.?\d+', text)
    if numbers:
        return float(numbers[-1])
    
    return None

# For demonstration, let's use a sample from GSM8K
sample_idx = 0  # You can change this to try different examples
sample = test_set[sample_idx]
question = sample["question"]
reference_answer = sample["answer"]

print(f"Question: {question}")
print(f"Reference answer: {reference_answer}")

# Create the prompt for AceMath
messages = [{"role": "user", "content": f"{question}Let's solve it step by step. You must give your final answer as an integer after '####'"}]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

# Generate answer
model_inputs = tokenizer([text], return_tensors="pt").to("cuda")
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512,
    temperature=0.1,  # Lower temperature for more deterministic outputs
    do_sample=True,
    top_p=0.95
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

print("\nAceMath Solution:")
print(response)

# Extract the numerical answer for evaluation
predicted_answer = extract_answer(response)
reference_numeric = extract_answer(reference_answer)

print(f"\nExtracted answer: {predicted_answer}")
print(f"Reference numeric answer: {reference_numeric}")

# Check if the answer is correct
if predicted_answer is not None and reference_numeric is not None:
    # Allow for small floating point differences
    if abs(predicted_answer - reference_numeric) < 1e-6:
        print("✓ Correct!")
    else:
        print("✗ Incorrect")
else:
    print("Could not extract a numeric answer")

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Question: Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?
Reference answer: Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.
She makes 9 * 2 = $<<9*2=18>>18 every day at the farmer’s market.
#### 18

AceMath Solution:
To determine how much Janet makes every day at the farmers' market, we need to follow these steps:

1. Calculate the number of eggs laid per day.
2. Determine the number of eggs consumed by Janet's family and friends.
3. Calculate the number of eggs remaining for sale.
4. Calculate the revenue from selling the remaining eggs.

**Step 1: Calculate the number of eggs laid per day.**

Janet's ducks lay 16 eggs per day.

**Step 2: Determine the number of eggs consumed by Janet's family and friends.**

Janet eats 3 eggs for breakfast every m